In [1]:
import pandas as pd
from sympy import isprime

df = pd.read_excel('task.xlsx', sheet_name='Tasks')
df = df[['num1', 'num2', 'num3', 'date1', 'date2', 'date3']]
df.head(5)

,num1,num2,num3,date1,date2,date3
0,Сколько четных чисел в этом столбце?,Сколько простых чисел в этом столбце?,"Сколько чисел, меньших 0.5 в этом столбце?",Столько вторников в этом столбце?,Сколько вторников в этом столбце?,Сколько последних вторников месяца в этом стол...
1,14,54,"0 ,481208997226 7124",Mon Aug 6 07:11:19 2018,2026-07-19 08:15:41.695463,10-04-2011
2,12,55,0 .9092218987487982,Thu Dec 1 01:32:03 2016,2006-08-06 16:18:39.172100,08-28-2006
3,25,2,0. 3268611158564687,Wed Aug 2 22:37:44 2028,2020-11-08 10:08:31.158667,11-21-2019
4,53,3,0. 38081302402434 647,Thu Feb 19 06:32:31 2009,2015-01-13 21:46:36.390139,06-26-2010


In [2]:
# Сколько четных чисел в num1?
num1_df = df[df['num1'].apply(pd.to_numeric, errors='coerce') % 2 == 0]
num1_df.shape[0]

507

In [3]:
# Сколько простых чисел в num2?
num2_df = df[df['num2'].apply(isprime) == True]
num2_df.shape[0]

168

In [4]:
# Сколько чисел меньших 0.5 в num3?
num3_df = df['num3']
# очищаю значения от пробелов и заменяем запятые на точки
num3_df = num3_df.str.replace(' ', '').replace(',', '.')
# преобразую строки в числа, оставляю только те значения, которые меньше 0.5
num3_df = num3_df[num3_df.apply(pd.to_numeric, errors='coerce') < 0.5]
num3_df.shape[0]

361

In [5]:
# Сколько вторников в date1?
date1_df = df[df['date1'].str.contains('Tue')]
date1_df.shape[0]

156

In [6]:
# Проверяю, что в date1 нет вторников, записанных с маленькой буквы
control_date1_df = df[df['date1'].str.contains('tue')]
control_date1_df

,num1,num2,num3,date1,date2,date3


In [13]:
# Сколько вторников в date2?
date2_df = pd.to_datetime(df['date2'][1:])
date2_df = date2_df[date2_df.dt.dayofweek == 1]
date2_df.shape[0]

138

In [14]:
# Сколько последних вторников месяца в date3?
date3_df = pd.to_datetime(df['date3'][1:])

# новый датафрейм для работы с вычисляемыми значениями
answer_df = pd.DataFrame()

# перекладываю даты из предыдущего датафрейма
answer_df['date'] = date3_df

# получаю столбец с последним днем текущего месяца
answer_df['mEnd'] = date3_df + pd.tseries.offsets.MonthEnd(1)

# получаю дату последнего вторника и оставляю только те строки, где текущая дата = дате последнего вторника
offset = (answer_df['mEnd'].dt.weekday - 1) % 7
answer_df['last_Tues'] = answer_df['mEnd'] - pd.to_timedelta(offset, unit='D')
answer_df = answer_df[answer_df['date'] == answer_df['last_Tues']]
answer_df.shape[0]

39

In [15]:
# Идеи для улучшения:
#     - написать класс на питоне, который будет принимать на вход .xlxs, а отдавать JSON с ответом. Класс содержит
#     методы определяющие ответ на каждый из поставленных вопросов и метод возвращающий ответ. Это позволит обрабатывать
#     большее количество подобных файлов, при необходимости.
#     - сделать простой веб сервер на Flask, который принимает .xlxs, а ответ отрисовывает в таблице
#     на отдельной странице
#     - упаковать решение в контейнер Docker